<a href="https://colab.research.google.com/github/tabaraei/Data-Science-Tips/blob/master/REST-API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Flask

In [ ]:
pip install flask

`api.py`:

In [ ]:
from flask import Flask, request, jsonify, render_template, redirect, url_for, abort
from werkzeug.exceptions import HTTPException
app = Flask(__name__)


# home screen
@app.route('/', methods=['GET'])
def home():
    return '<h1>Hello, World!</h1>'

# main screen
@app.route('/main', methods=['GET'])
def main():
    return render_template('main.html')

# form submit
@app.route('/api/form', methods=['POST'])
def form():
    return jsonify(request.form)

# {url}?arg1=value1&arg2=value2
@app.route('/api/params', methods=['POST', 'GET'])
def params():
    try: return jsonify(request.args)
    except: return redirect(url_for('page_not_found'))

# error handler
@app.errorhandler(404)
def page_not_found(e):
    return '<h1>404</h1><p>The resource could not be found.</p>', 404

# JSON
@app.route('/api/json', methods=['POST'])
def JSON():
    try: return jsonify(request.json)
    except: abort(400, description="error message")

# REST-API error handler
@app.errorhandler(Exception)
def error_handler(e):
    if isinstance(e, HTTPException):
        return jsonify(error=str(e)), e.code
    else:
        return jsonify(error=str(e)), 500


if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=8080)

`templates/main.html`:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <title>Signup</title>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/js/bootstrap.min.js"></script>
</head>

<body>

<div class="container">
  </br>
  <form class="form-horizontal" action="/api/form" method="post">

    <div class="form-group">
      <label class="control-label col-sm-2" for="name">Name:</label>
      <div class="col-sm-10">
        <input type="text" class="form-control" id="name" placeholder="Enter name" name="name" required>
      </div>
    </div>

    <div class="form-group">        
      <div class="col-sm-offset-2 col-sm-10">
        <div class="checkbox">
          <label><input type="checkbox" name="remember"> Remember me</label>
        </div>
      </div>
    </div>

    <div class="form-group">        
      <div class="col-sm-offset-2 col-sm-10">
        <button type="submit" class="btn btn-default">Submit</button>
      </div>
    </div>

  </form>
</div>

</body>
</html>

## Django

In [ ]:
pip install django djangorestframework

### 1- Create a virutal environment and install Django

In [ ]:
py -3.7 -m venv venv
.\venv\Scripts\activate
py -m pip install --upgrade pip
pip install django djangorestframework

### 2- Create a new project

In [ ]:
django-admin startproject myProject
django-admin startapp myApp

Cut all the contents of `myProject` into the current directory, then:

In [ ]:
py manage.py migrate
code .

Finally, go to `myProject/settings.py`, and add your application name, `myApp`, to the `INSTALLED_APPS`.

### 3- Run the project

In [ ]:
py manage.py runserver

### 4- Add API to the existing project

`myProject/settings.py`:

In [ ]:
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',

    'myApp',
    'rest_framework'  # add this line
]

`myProject/urls.py`:

In [ ]:
from django.contrib import admin
from django.urls import path, include   # addd include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('myApp.urls'))  # add this line
]

`myApp/views.py`:

In [ ]:
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework import status


@api_view(['GET', 'POST'])
def hello(request):
    try:
        if request.method == 'GET':
            return Response({'message': 'Hello, World!'})
        else:
            return Response({'message': f'Hello, {request.data["name"]}'})
    except:
        return Response({'error': 'Error happened'}, status=status.HTTP_400_BAD_REQUEST)

`myApp/urls.py` (Should be created):

In [ ]:
from django.urls import path
from myApp import views

urlpatterns = [
    path('hello', views.hello)
]

### 5- Add Model and Serializer:

`myApp/models.py`:

In [ ]:
from django.db import models

class YourClassName(models.Model):
  param1 = models.CharField(max_length=25)
  param2 = models.IntegerField()
  param3 = models.CharField(max_length=5,
            choices=(("abdce", "First Choice"),
                     ("edcba", "Second Choice")))

Run the following commands to create the model:

In [ ]:
py manage.py makemigrations
py manage.py migrate

`myApp/serializers.py`:

In [ ]:
from rest_framework import serilizers

class YourClassNameSerializer(serializers.Serializer):
  param1 = serializers.IntegerField(required=True)
  param2 = serializers.CharField(required=True, max_length=3)

`myApp/views.py`:

In [ ]:
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework import status
from .serializers import YourClassNameSerializer

@api_view(['POST'])
def func(request):
  ser = YourClassNameSerializer(data=request.data)
  if ser.is_valid():
    data = ser.data
  else:
    return Response(ser.errors, status=status.HTTP_400_BAD_REQUEST)